<font size=5>**Download and Preprocessing of Dataset for HiDRA_curve and ECHiDNA**</font>

<font size=4>Dataset for HiDRA_curve</font>
-Preprocessing: NCI60
-Finetuning: NCI-ALMANAC (monotherapy)

<font size=4>Dataset for ECHiDNA</font>
-NCI-ALMANAC (combination therapy)

<font size=4>Cell line expression data</font>
-CCLE

<font size=4>Drug information</font>
-PubChem

<font size=4>Pathway information</font>
-MSigDB

**The link for NCI60 dataset**
NCI60[https://wiki.nci.nih.gov/display/NCIDTPdata/NCI-60+Growth+Inhibition+Data] 
-file: DOSERESP.zip (CONCENTRATION/RESPONSE DATA)[https://wiki.nci.nih.gov/download/attachments/147193864/DOSERESP.zip?version=5&modificationDate=1664998650000&api=v2]

**The link for NCI-ALMANAC dataset**
NCI-ALMANAC[https://wiki.nci.nih.gov/display/NCIDTPdata/NCI-ALMANAC]
-file: ComboDrugGrowth_Nov2017.zip[https://wiki.nci.nih.gov/display/NCIDTPdata/NCI-ALMANACComboDrugGrowth_Nov2017.zip]

**The link for CCLE expression data**
DepMap[https://depmap.org/portal/]
-file: CCLE_RNAseq_rsem_genes_tpm_20180929.txt[https://depmap.org/portal/download/api/download?file_name=ccle%2Fccle_2019%2FCCLE_RNAseq_rsem_genes_tpm_20180929.txt.gz&bucket=depmap-external-downloads]

**The link for Pathway information**
MSigDB[http://www.gsea-msigdb.org/gsea/index.jsp]
-file: C2 KEGG[http://www.gsea-msigdb.org/gsea/msigdb/download_file.jsp?filePath=/msigdb/release/2022.1.Hs/c2.cp.kegg.v2022.1.Hs.symbols.gmt]



In [1]:
# Import libraries
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%config Completer.use_jedi=False
import os


import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols

#fix the random seed
np.random.seed(5)

import pickle
import random

base_directory=''

<font size=5>NCI60 (Monotherapy response dataset)</font>

In [2]:
NCI60_origin=pd.read_csv(base_directory+'Single/NCI60_March22/DOSERESP.csv')

#The csv file 'DOSERESP.csv' can be acquired by decompressing DOSERESP.zip

In [3]:
NCI60_origin=NCI60_origin[['NSC','CONCENTRATION','CELL_NAME','AVERAGE_GIPRCNT']]

In [4]:
NCI60_origin.CONCENTRATION=NCI60_origin.CONCENTRATION+6

#Viability unit: M (Molar)→μM (Micromolar)

In [5]:
NCI60_origin.AVERAGE_GIPRCNT=NCI60_origin.AVERAGE_GIPRCNT+100
NCI60_origin.AVERAGE_GIPRCNT=NCI60_origin.AVERAGE_GIPRCNT/100

#Viability range: (-100~100)→(0~2)

In [6]:
def cellname_convert(cellname):
    return (''.join(e for e in cellname if e.isalnum()))

#The function that removes special characters from the string

In [7]:
cellline_original=[cellname_convert(x) for x in NCI60_origin.CELL_NAME]

In [8]:
NCI60_origin['CELL_NAME_without_character']=cellline_original

In [10]:
NCI60_origin.to_csv(base_directory+'Single/NCI60_March22/220921_NCI60_raw.csv')

In [9]:
NCI60_new_CCLE=pd.read_csv(base_directory+'CellLine/NCI60March-CCLE_LookUpTable.csv',index_col=1)

#The csv file 'NCI60March-CCLE_LookUpTable.csv' is the look up table that matches NCI cell line name-CCLE cell line name
#The file was generated manually

In [11]:
NCI60_origin_Cell_valid=NCI60_origin[NCI60_origin.CELL_NAME_without_character.isin(NCI60_new_CCLE.index)]

In [12]:
NCI60_origin_CCLE=NCI60_new_CCLE.loc[NCI60_origin_Cell_valid['CELL_NAME_without_character']]

In [13]:
NCI60_origin_Cell_valid['CellName']=NCI60_origin_CCLE.CCLE_CellName.values

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
NCI60_origin_Cell_valid=NCI60_origin_Cell_valid[['NSC','CONCENTRATION','CellName','AVERAGE_GIPRCNT']]

In [15]:
NCI60_origin_Cell_valid.to_csv(base_directory+'Single/NCI60_March22/220921_NCI60_with_cellname.csv')

In [16]:
NCI60=pd.read_csv(base_directory+'Single/NCI60_March22/220921_NCI60_with_cellname.csv',index_col=0)

In [17]:
NCI60=NCI60.groupby(by=['NSC','CellName','CONCENTRATION']).median().reset_index()

#Normalization datapoint by grouping datapoints by median

In [19]:
NCI60=NCI60[NCI60.AVERAGE_GIPRCNT<3]

#Remove outliers

In [21]:
NCI60.to_csv(base_directory+'Single/NCI60_March22/220921_NCI60_median.csv')

<font size=5>Drug preprocessing (Monotherapy response dataset)</font>

In [22]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

suppl = Chem.SDMolSupplier(base_directory+'Single/NCI60_compound.sdf')
mols=[x for x in suppl]

#The file 'NCI60_compound.sdf' is the sdf file with NCI compound informations

[10:46:59] Explicit valence for atom # 4 O, 3, is greater than permitted
[10:46:59] ERROR: Could not sanitize molecule ending on line 736
[10:46:59] ERROR: Explicit valence for atom # 4 O, 3, is greater than permitted
[10:46:59] Explicit valence for atom # 7 Cl, 2, is greater than permitted
[10:46:59] ERROR: Could not sanitize molecule ending on line 25006
[10:46:59] ERROR: Explicit valence for atom # 7 Cl, 2, is greater than permitted
[10:46:59] Explicit valence for atom # 0 O, 3, is greater than permitted
[10:46:59] ERROR: Could not sanitize molecule ending on line 27712
[10:46:59] ERROR: Explicit valence for atom # 0 O, 3, is greater than permitted
[10:46:59] Explicit valence for atom # 1 O, 3, is greater than permitted
[10:46:59] ERROR: Could not sanitize molecule ending on line 34694
[10:46:59] ERROR: Explicit valence for atom # 1 O, 3, is greater than permitted
[10:46:59] Explicit valence for atom # 2 O, 3, is greater than permitted
[10:46:59] ERROR: Could not sanitize molecule e

In [23]:
mols_without_None=[x for x in mols if x!=None ]

#Filtering 'None' molecules (molecule that were not processible with rdkit library)

In [24]:
Mol_df=pd.DataFrame({'Mol':mols_without_None,'NSC':[x.GetProp('_Name') for x in mols_without_None]})

#Dataframe with Molecules

In [25]:
Mol_df['fingerprint']=[AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=512) for mol in Mol_df.Mol]

#Get Morgan Fingerprint, that will be used as drug feature

In [26]:
compound_morgan512=pd.DataFrame([list(x) for x in Mol_df.fingerprint.values])

#Dataframe that consists of Morgan Fingerprints

In [27]:
compound_morgan512.index=Mol_df['NSC']

#Index mapping

In [34]:
compound_morgan512.to_csv(base_directory+'Drug/NCI60_compound_Morgan512.csv')

In [35]:
compound_morgan512=pd.read_csv(base_directory+'Drug/NCI60_compound_Morgan512.csv',index_col=0)

In [36]:
NSC_list=compound_morgan512.index

#NSC_list=Drug list

<font size=5>Cell line preprocessing (Monotherapy response dataset)</font>

In [37]:
CellLine_expression=pd.read_table(base_directory+'CCLE_RNAseq_rsem_genes_tpm_20180929.txt')

#Cell line expression information

In [38]:
CellLine_expression=CellLine_expression.transpose()

In [39]:
gene_name=CellLine_expression.iloc[0]

#gene_name=Gene list (Gene name in Ensembl ID)

In [40]:
gene_name=[x.split('.')[0] for x in gene_name]

#The example of Gene name is 'ENSG00000000003.10', so using the string before '.' only

In [41]:
CellLine_expression.columns=gene_name
CellLine_expression=CellLine_expression.iloc[2:]

#First row (iloc 0) is Gene name in Ensembl ID
#Second row (iloc 1) is Transcriptome name in Ensembl ID
#From third row, the expression data

In [42]:
CellLine_expression.index=[x.split('_')[0] for x in CellLine_expression.index]

#The index, Cell line name is in the format of 'CellName_Organ', so using the string before '_' only

In [43]:
import mygene
  
mg=mygene.MyGeneInfo()
ginfo=mg.querymany(gene_name,scopes='ensembl.gene') 

gene_id_list=[x['query'] for x in ginfo]
 
gene_symbol_list=[]
for idx,x in enumerate(ginfo):
    if 'symbol' in x.keys():
        gene_symbol_list.append(x['symbol'])
    else:
        gene_symbol_list.append(np.NaN)

#Mapping Ensembl ID with gene symbol by using mygene API

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [45]:
gene_info=pd.DataFrame({'Gene_ID':gene_id_list,'Gene_Symbol':gene_symbol_list})
gene_info=gene_info.dropna()
gene_info=gene_info.reset_index(drop=True)
gene_info.index=gene_info.Gene_ID
gene_info=gene_info[['Gene_Symbol']]

#The lookup table dataframe Gene symbol↔Ensembl ID

In [46]:
valid_ENSG_list=list(gene_info.index)
CellLine_expression=CellLine_expression[valid_ENSG_list]
CellLine_expression.columns=gene_info.Gene_Symbol.values
CellLine_expression.to_csv(base_directory+'CellLine/CCLE_expression.csv')

#Using expression of valid genes only

In [48]:
def log2plus1(x):
    return np.log2(x+1)

#Function for log normalization

In [49]:
CellLine_expression_float=CellLine_expression.astype('float')
CellLine_expression_log2=log2plus1(CellLine_expression_float)
CellLine_expression_log2.to_csv(base_directory+'CellLine/CCLE_expression_log2.csv')

#Expression normalized by Log2 normalization

In [86]:
from scipy.stats import zscore
CellLine_expression_zscore=CellLine_expression_log2.apply(zscore,axis=1)

#Z-score normalization

In [87]:
CellLine_expression_zscore=CellLine_expression_zscore.groupby(CellLine_expression_zscore.index).median()
CellLine_expression_zscore=CellLine_expression_zscore.transpose()
CellLine_expression_zscore=CellLine_expression_zscore.groupby(CellLine_expression_zscore.index).median()
CellLine_expression_zscore=CellLine_expression_zscore.transpose()
CellLine_expression_zscore.to_csv(base_directory+'CellLine/CCLE_expression_zscore.csv')

#Median normalization as there are two cell line expression for one cell line
#Median normalization as there are some cases that two or more rows for one gene

In [93]:
CellLine_expression_zscore=pd.read_csv(base_directory+'CellLine/CCLE_expression_zscore.csv',index_col=0)

In [114]:
valid_gene_list=CellLine_expression_zscore.columns
#Loading Gene Set
#Gene Set File (gmt) is 'KEGG subset of CP' from MSigDB (http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp)
GeneSet_List=[]
GeneSetFile=base_directory+'c2.cp.kegg.v7.5.1.symbols.gmt'
with open(GeneSetFile) as f:
    reader = csv.reader(f)
    data = list(list(rec) for rec in csv.reader(f, delimiter='\t')) #reads csv into a list of lists
    for row in data:
        GeneSet_List.append(row)

GeneSet_Dic={}
for GeneSet in GeneSet_List:
    GeneSet_Dic[GeneSet[0]]=GeneSet[2:]

#Delete genes that are not valid
#In here, E3 is just a name of one of cell line that is valid
GeneSet_Dic_withoutNA={}
for GeneSet in GeneSet_Dic:
    GeneSet_tmp=pd.Series(GeneSet_Dic[GeneSet])
    GeneSet_tmp=GeneSet_tmp[GeneSet_tmp.isin(valid_gene_list)]
    GeneSet_Dic_withoutNA[GeneSet]=GeneSet_tmp


In [116]:
def CelllineFeatureExtract(ExpressionMatrix, GeneSetDic, CellLine):
    X_Feature=[]
    for GeneSet in GeneSetDic.keys():
        Gene_in_GeneSet=[]
        for Gene in GeneSetDic[GeneSet]:
            Gene_in_GeneSet.append(Gene)
        X_Feature.append(ExpressionMatrix[Gene_in_GeneSet].loc[[CellLine]])
    return X_Feature

#The function for the cell line feature extraction (Formatting the expression into gene set forms)

In [117]:
cellline_input=[]
for i in range(len(GeneSet_Dic_withoutNA)):
    cellline_input.append(pd.DataFrame())
for cellline in CellLine_expression_zscore.index:
    print(cellline)
    x=CelllineFeatureExtract(CellLine_expression_zscore,GeneSet_Dic_withoutNA,cellline)
    for j in range(len(GeneSet_Dic_withoutNA)):
        cellline_input[j]=cellline_input[j].append(x[j])


22RV1
2313287
253J
253JBV
42MGBA
5637
59M
639V
647V
697
769P
786O
8305C
8505C
8MGBA
A101D
A1207
A172
A204
A2058
A253
A2780
A375
A3KAW
A427
A498
A4FUK
A549
A673
A704
ABC1
ACCMESO1
ACHN
AGS
ALLSIL
AM38
AML193
AMO1
AN3CA
ASPC1
AU565
BC3C
BCP1
BCPAP
BDCM
BEN
BFTC905
BFTC909
BHT101
BHY
BICR16
BICR18
BICR22
BICR31
BICR56
BICR6
BL41
BL70
BT12
BT20
BT474
BT483
BT549
BV173
BXPC3
C2BBE1
C32
C8166
CA46
CACO2
CADOES1
CAKI1
CAKI2
CAL120
CAL12T
CAL148
CAL27
CAL29
CAL33
CAL51
CAL54
CAL62
CAL78
CAL851
CALU1
CALU3
CALU6
CAMA1
CAOV3
CAOV4
CAPAN1
CAPAN2
CAS1
CCFSTTG1
CCK81
CFPAC1
CH157MN
CHAGOK1
CHP126
CHP212
CI1
CJM
CL11
CL14
CL34
CL40
CMK
CMLT1
COLO201
COLO320
COLO668
COLO678
COLO679
COLO680N
COLO684
COLO741
COLO783
COLO792
COLO800
COLO829
CORL105
CORL23
CORL24
CORL279
CORL311
CORL47
CORL88
CORL95
COV318
COV362
COV434
COV644
CW2
D283MED
D341MED
DANG
DAOY
DAUDI
DB
DBTRG05MG
DEL
DETROIT562
DKMG
DM3
DMS114
DMS153
DMS273
DMS454
DMS53
DMS79
DND41
DOHH2
DU145
DU4475
DV90
EB1
EBC1
ECC10
ECC12
ECGI10
EFE184
EF

In [120]:
for idx,df in enumerate(cellline_input):
    df.to_csv(base_directory+'CellLine/Input/'+str(idx)+'.csv')


<font size=5>NCI60 filtering and preprocessing</font>

In [126]:
cellline_list=CellLine_expression_zscore.index

In [127]:
NCI60_valid=NCI60[(NCI60.NSC.isin(NSC_list))&(NCI60.CellName.isin(cellline_list))]

In [128]:
NCI60_valid

,NSC,CellName,CONCENTRATION,AVERAGE_GIPRCNT
0,1,786O,-2.0,1.916247
1,1,786O,-1.0,2.006393
2,1,786O,0.0,2.069248
3,1,786O,1.0,2.066457
4,1,786O,2.0,0.079399
...,...,...,...,...
11255211,788948,UO31,-2.0,1.878057
11255212,788948,UO31,-1.0,1.869322
11255213,788948,UO31,0.0,1.881901
11255214,788948,UO31,1.0,1.846261


In [131]:
NCI60_valid_sorted=NCI60_valid.sort_values(by=['NSC','CellName','CONCENTRATION'])

In [133]:
NCI60_valid_sorted['delta']=NCI60_valid_sorted.AVERAGE_GIPRCNT.shift(1)-NCI60_valid_sorted.AVERAGE_GIPRCNT

In [134]:
NCI60_valid_sorted['identity']=(NCI60_valid_sorted.NSC.shift(1)==NCI60_valid_sorted.NSC)&(NCI60_valid_sorted.CellName.shift(1)==NCI60_valid_sorted.CellName)

In [135]:
min_df=pd.DataFrame(NCI60_valid.groupby(by=['NSC','CellName']).min().AVERAGE_GIPRCNT)
min_df.columns=['Viability_min']
min_df=min_df.reset_index()

In [136]:
NCI60_valid_filtered=pd.merge(NCI60_valid_sorted,min_df,on=['NSC','CellName'],how='left')
NCI60_valid_filtered=NCI60_valid_filtered[NCI60_valid_filtered.Viability_min<1.5]

In [138]:
filtered_df=NCI60_valid_filtered[(NCI60_valid_filtered.identity==True)&(NCI60_valid_filtered.delta<=-0.5)][['NSC','CellName']]
filtered_df['is_filtered']=True
filtered_df=filtered_df.drop_duplicates()
NCI60_valid_filtered=pd.merge(NCI60_valid_filtered,filtered_df,on=['NSC','CellName'],how='left')

In [140]:
NCI60_valid_filtered=NCI60_valid_filtered[NCI60_valid_filtered.is_filtered!=True][['NSC','CellName','CONCENTRATION','AVERAGE_GIPRCNT']]

In [142]:
NCI60_valid_filtered.to_csv(base_directory+'Single/NCI60_March22/220922_NCI60_filtered_by_delta_and_y_min.csv')

<font size=5>NCI60 Parameter calculation</font>

In [143]:
identifier_df=NCI60_valid_filtered[['CellName','NSC']]
identifier_df=identifier_df.drop_duplicates()

#identifier_df: The list df of 'Cell line-drug pair'

In [144]:
identifier_df

,CellName,NSC
0,786O,1
6,A498,1
12,A549,1
18,ACHN,1
24,BT549,1
...,...,...
6092738,SW620,783538
6092743,T47D,783538
6092748,UACC257,783538
6092753,UACC62,783538


In [129]:
def response(x,y_max,y_min,IC50,slope):
    return (y_min+((y_max-y_min)/(1+np.exp(slope*(x-IC50)))))

#The function for the response curve

In [130]:
import scipy
def cal_parameter(NSC,CellLine):
    df_target=NCI60_valid[(NCI60_valid.NSC==NSC)&(NCI60_valid.CellName==CellLine)].sort_values(by='CONCENTRATION')
    x_list=df_target.CONCENTRATION.values[0:]
    y_list=df_target.AVERAGE_GIPRCNT.values[0:]
    param=scipy.optimize.curve_fit(response,x_list,y_list,bounds=([1.9,0,-5,1],[2.1,2,5,3]))[0] #param=[y_max,y_min,IC50,slope]
    return param

#The function for the parameter calculation


In [145]:
identifier_df['y_max']=None
identifier_df['y_min']=None
identifier_df['IC50']=None
identifier_df['slope']=None

In [146]:
import multiprocessing
from multiprocessing import Pool

num_cores = 40

In [147]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [148]:
from scipy.optimize import OptimizeWarning


In [149]:
def func_dataframe(dataframe):
    y_max_list=[]
    y_min_list=[]
    IC50_list=[]
    slope_list=[]
    for idx,x in dataframe.iterrows():
        try:
            params=cal_parameter(int(x.NSC),x.CellName)
        except RuntimeError:
            params=[None,None,None,None]
        except OptimizeWarning:
            params=[None,None,None,None]
            
        y_max_list.append(params[0])
        y_min_list.append(params[1])
        IC50_list.append(params[2])
        slope_list.append(params[3])
    dataframe['y_max']=y_max_list
    dataframe['y_min']=y_min_list
    dataframe['IC50']=IC50_list
    dataframe['slope']=slope_list
    
    return dataframe

In [150]:
parameter_df=parallelize_dataframe(identifier_df,func_dataframe)

/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning

In [317]:
parameter_df.to_csv(base_directory+'Single/NCI60_March22/220922_NCI60_median_parameter_raw.csv')

In [320]:
parameter_df_filtered=parameter_df.dropna()

In [328]:
parameter_df_filtered=parameter_df_filtered[(parameter_df_filtered.IC50>-4.99)&(parameter_df_filtered.IC50<4.99)]

#Filter the overfitted datapoint 

In [335]:
parameter_df_filtered.to_csv(base_directory+'Single/NCI60_March22/220923_NCI60_median_parameter_filtered.csv')

In [332]:
NCI60=pd.merge(NCI60_valid,parameter_df_filtered,how='inner',on=['NSC','CellName'])

In [336]:
NCI60.to_csv(base_directory+'Single/NCI60_March22/220923_NCI60_median_filtered.csv')

<font size=5>NCI-ALMANAC (combination therapy response)</font>

In [337]:
NCI_ALMANAC=pd.read_csv(base_directory+'Synergy/NCI-ALMANAC.csv',index_col=0)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [339]:
NCI_ALMANAC=NCI_ALMANAC[['NSC1','CONC1','NSC2','CONC2','CELLNAME','PERCENTGROWTH']]

<font size=3>NCI-ALMANAC_mono (monotherapy from NCI-ALMANAC)</font>

In [340]:
NCI_ALMANAC_mono=NCI_ALMANAC[NCI_ALMANAC.NSC2.isna()]

In [342]:
NCI_ALMANAC_mono.iloc[812960,4]='SF-539'

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [345]:
import re

NCI_ALMANAC_mono['CellName_without_character']=[re.sub(r"[^a-zA-Z0-9]","",x) for x in NCI_ALMANAC_mono.CELLNAME]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [346]:
NCI_ALMANAC_mono

,NSC1,CONC1,NSC2,CONC2,CELLNAME,PERCENTGROWTH,CellName_without_character
COMBODRUGSEQ,,,,,,,
260667,752,1.000000e-07,NaN,NaN,786-0,92.670,7860
260668,752,1.000000e-06,NaN,NaN,786-0,78.366,7860
260669,752,1.000000e-05,NaN,NaN,786-0,29.829,7860
260832,3088,1.000000e-06,NaN,NaN,786-0,97.488,7860
260833,3088,1.000000e-05,NaN,NaN,786-0,94.022,7860
...,...,...,...,...,...,...,...
5732421,681239,1.000000e-08,NaN,NaN,SF-539,-51.889,SF539
5732422,681239,3.170000e-08,NaN,NaN,SF-539,-61.133,SF539
5732423,681239,1.000000e-07,NaN,NaN,SF-539,-62.724,SF539


In [347]:
LookUpTable=pd.read_csv(base_directory+'CellLine/NCI60March-CCLE_LookUpTable.csv',index_col=1)

In [348]:
LookUpTable

,Unnamed: 0,CCLE_CellName
CELL_NAME,,
7860,0,786O
769P,396705,769P
A172HFine,2094,A172
A204,8590,A204
A498,6,A498
...,...,...
UACC62,347,UACC62
UO31,353,UO31
WIDR,1692477,HT29


In [352]:
NCI_ALMANAC_mono=NCI_ALMANAC_mono[NCI_ALMANAC_mono.CellName_without_character.isin(LookUpTable.index)]

In [353]:
NCI_ALMANAC_mono['CellName']=LookUpTable.loc[NCI_ALMANAC_mono.CellName_without_character].CCLE_CellName.values

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [354]:
NCI_ALMANAC_mono=NCI_ALMANAC_mono[['NSC1','CONC1','CellName','PERCENTGROWTH']]

In [355]:
NCI_ALMANAC_mono_median=NCI_ALMANAC_mono.groupby(['NSC1','CONC1','CellName']).median()

In [356]:
NCI_ALMANAC_mono_median=NCI_ALMANAC_mono_median.reset_index()
NCI_ALMANAC_mono_median=NCI_ALMANAC_mono_median[['NSC1','CONC1','CellName','PERCENTGROWTH']]
NCI_ALMANAC_mono_median.columns=['NSC','CONC','CellName','PERCENTGROWTH']
NCI_ALMANAC_mono_median.PERCENTGROWTH+=100
NCI_ALMANAC_mono_median.PERCENTGROWTH/=100

In [357]:
NCI_ALMANAC_mono_median.CONC=np.log10(NCI_ALMANAC_mono_median.CONC)
NCI_ALMANAC_mono_median.CONC+=6

In [358]:
NCI_ALMANAC_mono_median

,NSC,CONC,CellName,PERCENTGROWTH
0,740,-3.0,786O,2.06265
1,740,-3.0,A498,2.03900
2,740,-3.0,A549,2.01855
3,740,-3.0,ACHN,2.01810
4,740,-3.0,BT549,1.99460
...,...,...,...,...
37654,763371,2.0,SW620,1.06824
37655,763371,2.0,T47D,1.14966
37656,763371,2.0,UACC257,0.94380
37657,763371,2.0,UACC62,1.05890


In [361]:
identifier_df=NCI_ALMANAC_mono_median[['CellName','NSC']]
identifier_df=identifier_df.drop_duplicates()

In [362]:
identifier_df['y_min']=None
identifier_df['IC50']=None
identifier_df['slope']=None

In [363]:
import scipy
#Parameter calculation
def cal_parameter(NSC,CellLine):
    df_target=NCI_ALMANAC_mono_median[(NCI_ALMANAC_mono_median.NSC==NSC)&(NCI_ALMANAC_mono_median.CellName==CellLine)].sort_values(by='CONC')
    x_list=df_target.CONC.values[0:]
    y_list=df_target.PERCENTGROWTH.values[0:]
    param=scipy.optimize.curve_fit(response,x_list,y_list,bounds=([0,-5,1],[2,5,3]))[0] #param=[y_min,IC50,slope]
    return param

In [364]:
import multiprocessing
from multiprocessing import Pool

num_cores = 40

In [368]:
def func_dataframe(dataframe):
    y_min_list=[]
    y_max_list=[]
    IC50_list=[]
    slope_list=[]
    for idx,x in dataframe.iterrows():
        try:
            params=cal_parameter(int(x.NSC),x.CellName)
        except RuntimeError:
            params=[None,None,None,None]
        except OptimizeWarning:
            params=[None,None,None]
            
        y_min_list.append(params[0])
        IC50_list.append(params[1])
        slope_list.append(params[2])
    dataframe['y_min']=y_min_list
    dataframe['IC50']=IC50_list
    dataframe['slope']=slope_list
    
    return dataframe

In [369]:
tmp=parallelize_dataframe(identifier_df,func_dataframe)

/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning

In [370]:
tmp

,CellName,NSC,y_min,IC50,slope
0,786O,740,1.183439e+00,-2.282372,3.000000
1,A498,740,1.250966e+00,-1.450167,3.000000
2,A549,740,1.142098e+00,-1.638116,3.000000
3,ACHN,740,1.120243e+00,-1.343586,3.000000
4,BT549,740,1.214710e+00,-0.522382,1.947413
...,...,...,...,...,...
37476,UACC62,763371,5.686226e-17,2.062063,3.000000
37477,UO31,763371,5.444916e-22,1.968213,1.748182
37485,EKVX,763371,1.338333e-08,4.069997,1.000000
37489,HOP92,763371,1.579005e-14,1.896235,3.000000


In [376]:
tmp.to_csv(base_directory+'Single/NCI_ALMANAC_mono/220923_NCI_ALMANAC_median_parameter_raw.csv')

In [377]:
identifier_df_filtered=tmp.dropna()

In [378]:
identifier_df_filtered=identifier_df_filtered[(identifier_df_filtered.IC50>-4.99)&(identifier_df_filtered.IC50<4.99)]

In [379]:
identifier_df_filtered.to_csv(base_directory+'Single/NCI_ALMANAC_mono/220923_NCI_ALMANAC_median_parameter_filtered.csv')

In [380]:
identifier_df_filtered

,CellName,NSC,y_min,IC50,slope
0,786O,740,1.183439e+00,-2.282372,3.000000
1,A498,740,1.250966e+00,-1.450167,3.000000
2,A549,740,1.142098e+00,-1.638116,3.000000
3,ACHN,740,1.120243e+00,-1.343586,3.000000
4,BT549,740,1.214710e+00,-0.522382,1.947413
...,...,...,...,...,...
37476,UACC62,763371,5.686226e-17,2.062063,3.000000
37477,UO31,763371,5.444916e-22,1.968213,1.748182
37485,EKVX,763371,1.338333e-08,4.069997,1.000000
37489,HOP92,763371,1.579005e-14,1.896235,3.000000


In [381]:
NCI_ALMANAC=pd.merge(NCI_ALMANAC_mono_median,identifier_df_filtered,how='inner',on=['NSC','CellName'])

In [382]:
NCI_ALMANAC.to_csv(base_directory+'Single/NCI_ALMANAC_mono/220923_NCI_ALMANAC_mono_median_filtered.csv')